# Marketing Campaign Creatives Generation Agent (100% Open-Source)

Interactive LangGraph-based AI Agent for Marketing Content Generation using **state-of-the-art open-source models** - no API keys required!

This notebook implements an intelligent agent that:
- Takes short user inputs and expands them into detailed prompts using **Qwen2.5-7B-Instruct** (local)
- Generates promotional images using **FLUX.1-dev** (Black Forest Labs) - Best quality open-source image model
- Generates promotional videos using **LTX-Video** (Lightricks) - Fast, real-time video generation
- Generates voice-over narration using **Kokoro-82M** TTS (Hugging Face)
- Allows iterative feedback and refinement

**Requirements:**
- Google Colab with A100 GPU (40GB VRAM)
- No API keys needed - everything runs locally!

## Installing the Necessary Libraries

In [ ]:
# Install Python dependencies
!pip install -q -U \
    langgraph \
    langchain-core \
    diffusers \
    transformers \
    accelerate \
    torch \
    torchao \
    safetensors \
    sentencepiece \
    moviepy \
    soundfile \
    imageio-ffmpeg \
    bitsandbytes \
    kokoro>=0.9.2

# Install system dependencies for TTS and video processing
!apt-get -qq -y install espeak-ng ffmpeg

**Note:** After running the above cell, restart the runtime and run all cells sequentially from the next cell.

## Importing Libraries

In [ ]:
# Transformers (for local LLM)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# LangGraph Libraries
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# LangChain Core
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# Type Annotations
from typing import TypedDict, Annotated, Literal, Optional, List, Any

# PyTorch and utilities
import torch
import gc
import os
import sys
import time
import matplotlib.pyplot as plt

## Configuration and Client Setup

In [ ]:
# Model Configuration - ALL LOCAL, NO API KEYS!
LLM_MODEL = "Qwen/Qwen2.5-7B-Instruct"  # Agent LLM - excellent for instruction following
IMAGE_MODEL = "black-forest-labs/FLUX.1-dev"  # State-of-the-art image generation
VIDEO_MODEL = "Lightricks/LTX-Video"  # Fast real-time video generation
TTS_MODEL = "hexgrad/Kokoro-82M"  # Hugging Face TTS model

# Generation Configuration
MAX_ITERATIONS = 10  # Maximum feedback iterations
IMAGE_WIDTH = 1024  # FLUX.1 output width
IMAGE_HEIGHT = 1024  # FLUX.1 output height
IMAGE_STEPS = 28  # FLUX.1-dev recommended steps (20-50)
VIDEO_WIDTH = 768  # LTX-Video width (must be divisible by 32)
VIDEO_HEIGHT = 512  # LTX-Video height (must be divisible by 32)
VIDEO_FRAMES = 241  # LTX-Video frames (must be divisible by 8 + 1, e.g., 241 = 30*8+1 ~10 sec at 24fps)
VIDEO_FPS = 24  # LTX-Video outputs at 24 FPS
DEFAULT_VOICE = "af_heart"  # American Female (Heart) voice for Kokoro

# LLM Generation Settings
LLM_MAX_NEW_TOKENS = 1024
LLM_TEMPERATURE = 0.7

# Global variables for pre-loaded models
llm_model = None
llm_tokenizer = None

print(f"Configuration loaded - 100% LOCAL, NO API KEYS!")
print(f"  - Agent LLM: {LLM_MODEL}")
print(f"  - Image Model: {IMAGE_MODEL} (FLUX.1-dev)")
print(f"  - Video Model: {VIDEO_MODEL} (LTX-Video)")
print(f"  - TTS Model: {TTS_MODEL}")
print(f"  - Video Duration: ~{VIDEO_FRAMES/VIDEO_FPS:.1f} seconds ({VIDEO_FRAMES} frames at {VIDEO_FPS} FPS)")

## Memory Management Utility

In [ ]:
def clear_gpu_memory():
    """Clear GPU memory between model loads."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print(f"  GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


def get_gpu_memory_usage():
    """Get current GPU memory usage."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        return f"Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB"
    return "GPU not available"


# Check GPU availability
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Video generation will be very slow or may fail.")

## Pre-Download and Cache Models

**Important:** Run this cell to download all models before starting the agent. This ensures no download delays during the interactive session. Models are cached locally after first download.

In [ ]:
# ============================================================
# PRE-DOWNLOAD ALL MODELS
# ============================================================
# This cell downloads and caches all models so they're ready
# when the agent starts. No network delays during generation!
# ============================================================

print("="*60)
print("   DOWNLOADING AND CACHING MODELS")
print("   This may take 15-30 minutes on first run...")
print("   (FLUX.1-dev and LTX-Video are large models)")
print("="*60)

# --- 0. Download Qwen2.5-7B-Instruct (Agent LLM) ---
print("\n[0/4] Downloading Qwen2.5-7B-Instruct for agent reasoning...")
print(f"      Model: {LLM_MODEL}")
print("      Using 4-bit quantization to save memory...")
try:
    # 4-bit quantization config for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )

    llm_tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("      Qwen2.5-7B-Instruct loaded and ready!")
    print(f"      {get_gpu_memory_usage()}")
except Exception as e:
    print(f"      Error loading LLM: {e}")
    llm_model = None
    llm_tokenizer = None

# --- 1. Download FLUX.1-dev (Image Generation) ---
print("\n[1/4] Downloading FLUX.1-dev for image generation...")
print(f"      Model: {IMAGE_MODEL}")
print("      (This is a large model ~12GB, may take a few minutes)")
try:
    from diffusers import FluxPipeline

    image_pipe = FluxPipeline.from_pretrained(
        IMAGE_MODEL,
        torch_dtype=torch.bfloat16  # FLUX works best with bfloat16
    )
    # Keep on CPU initially to save memory
    print("      FLUX.1-dev loaded and ready!")
    print(f"      {get_gpu_memory_usage()}")
except Exception as e:
    print(f"      Error loading FLUX.1-dev: {e}")
    image_pipe = None

# --- 2. Download Kokoro TTS (Voice-over) ---
print("\n[2/4] Downloading Kokoro TTS for voice-over...")
print(f"      Model: {TTS_MODEL}")
try:
    from kokoro import KPipeline
    tts_pipe = KPipeline(lang_code='a')  # 'a' for American English
    print("      Kokoro TTS loaded and ready!")
except Exception as e:
    print(f"      Error loading Kokoro TTS: {e}")
    tts_pipe = None

# Clear GPU memory before loading the video model
clear_gpu_memory()

# --- 3. Download LTX-Video (Video Generation) ---
print("\n[3/4] Downloading LTX-Video for video generation...")
print(f"      Model: {VIDEO_MODEL}")
print("      (Fast real-time video generation model)")
try:
    from diffusers import LTXPipeline

    video_pipe = LTXPipeline.from_pretrained(
        VIDEO_MODEL,
        torch_dtype=torch.bfloat16  # LTX-Video works best with bfloat16
    )
    # Keep on CPU initially to save memory
    print("      LTX-Video loaded and ready!")
except Exception as e:
    print(f"      Error loading LTX-Video: {e}")
    video_pipe = None

# Clear some GPU memory
clear_gpu_memory()

print("\n" + "="*60)
print("   ALL MODELS DOWNLOADED AND CACHED!")
print("   You can now run the agent without download delays.")
print("="*60)
print(f"\nFinal GPU state: {get_gpu_memory_usage()}")

## Agent State Schema Definition

In [ ]:
class AgentState(TypedDict):
    """
    State schema for the marketing content generation agent.
    Extended with voice-over support.
    """
    # Message history
    messages: Annotated[List[BaseMessage], add_messages]

    # User inputs
    user_input: str
    feedback: Optional[str]

    # Expanded prompts
    expanded_image_prompt: Optional[str]
    expanded_video_prompt: Optional[str]
    expanded_voiceover_script: Optional[str]  # NEW: Script for TTS

    # Generation configuration
    generation_mode: Optional[Literal["image", "video", "both"]]
    include_voiceover: bool  # NEW: Whether to generate voice-over
    voice_style: Optional[str]  # NEW: Kokoro voice style

    # Generated outputs
    generated_image: Optional[Any]  # PIL Image object
    generated_video_path: Optional[str]
    generated_audio_path: Optional[str]  # NEW: Path to voice-over audio
    final_video_path: Optional[str]  # NEW: Video with audio combined

    # Control flow
    iteration_count: int
    is_satisfied: bool

## Tool Implementations

### Tool 1: Expand Prompt (Qwen2.5-7B Local)

In [ ]:
def parse_expanded_prompts(response_text: str) -> tuple:
    """
    Parse the response from LLM into separate prompts.
    Returns: (image_prompt, video_prompt, voiceover_script)
    """
    image_prompt = ""
    video_prompt = ""
    voiceover_script = ""

    # Split by markers
    if "IMAGE_PROMPT:" in response_text:
        parts = response_text.split("VIDEO_PROMPT:")
        image_prompt = parts[0].replace("IMAGE_PROMPT:", "").strip()

        if len(parts) > 1:
            video_parts = parts[1].split("VOICEOVER_SCRIPT:")
            video_prompt = video_parts[0].strip()

            if len(video_parts) > 1:
                voiceover_script = video_parts[1].strip()

    return image_prompt, video_prompt, voiceover_script


def generate_llm_response(messages: list) -> str:
    """
    Generate a response using the local Qwen model.
    Uses the global llm_model and llm_tokenizer.
    """
    global llm_model, llm_tokenizer

    if llm_model is None or llm_tokenizer is None:
        raise RuntimeError("LLM model not loaded. Run the model pre-download cell first.")

    # Format messages for Qwen chat template
    text = llm_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize
    model_inputs = llm_tokenizer([text], return_tensors="pt").to(llm_model.device)

    # Generate
    with torch.no_grad():
        generated_ids = llm_model.generate(
            **model_inputs,
            max_new_tokens=LLM_MAX_NEW_TOKENS,
            temperature=LLM_TEMPERATURE,
            do_sample=True,
            pad_token_id=llm_tokenizer.eos_token_id
        )

    # Decode only the new tokens
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = llm_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response


def expand_prompt(state: AgentState) -> dict:
    """
    Uses Qwen2.5-7B (local) to expand a short user input into detailed prompts
    for image generation, video generation, and voice-over.
    """
    user_input = state["user_input"]

    system_prompt = """You are an expert at creating detailed prompts for AI image and video generation.
Given a short description of promotional material needed, create THREE detailed prompts:

1. IMAGE PROMPT: A detailed description for generating a promotional poster/image.
   Include: visual style, colors, composition, mood, and specific details.
   NOTE: Do NOT request text in the image - AI image models struggle with text rendering.
   Focus on visual elements, products, atmosphere, and branding colors.

2. VIDEO PROMPT: A detailed description for generating a 6-second promotional video.
   Include: scene descriptions, camera movements, pacing, mood.
   CRITICAL RULE: Do NOT include any people, humans, faces, hands, baristas, customers,
   or any human figures in the video prompt. Focus ONLY on:
   - Products (food, drinks, items)
   - Environment/atmosphere (interior, exterior, decor)
   - Objects and textures (cups, tables, steam, etc.)
   - Abstract visuals and motion

3. VOICEOVER SCRIPT: A short, punchy voice-over script (max 40 words) for the promotional video.
   Include: tagline, key selling point, call-to-action.
   Keep it natural and conversational.

Format your response EXACTLY as:
IMAGE_PROMPT: [detailed image prompt here]
VIDEO_PROMPT: [detailed video prompt here - NO PEOPLE]
VOICEOVER_SCRIPT: [short script for TTS]
"""

    print("\nExpanding prompts with Qwen2.5-7B...")

    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"User request: {user_input}"}
        ]

        response_text = generate_llm_response(messages)
        image_prompt, video_prompt, voiceover_script = parse_expanded_prompts(response_text)

        print("\n" + "-"*50)
        print("EXPANDED PROMPTS")
        print("-"*50)
        print(f"\nImage Prompt Preview:\n{image_prompt[:200]}..." if len(image_prompt) > 200 else f"\nImage Prompt:\n{image_prompt}")
        print(f"\nVideo Prompt Preview:\n{video_prompt[:200]}..." if len(video_prompt) > 200 else f"\nVideo Prompt:\n{video_prompt}")
        print(f"\nVoiceover Script:\n{voiceover_script}")

        return {
            "expanded_image_prompt": image_prompt,
            "expanded_video_prompt": video_prompt,
            "expanded_voiceover_script": voiceover_script,
            "messages": [AIMessage(content="Expanded your request into detailed prompts.")]
        }

    except Exception as e:
        print(f"Error expanding prompts: {str(e)}")
        return {
            "messages": [AIMessage(content=f"Error expanding prompts: {str(e)}")]
        }

### Tool 2: Generate Poster (FLUX.1-dev)

In [ ]:
def generate_poster(state: AgentState) -> dict:
    """
    Generates a promotional image using pre-loaded FLUX.1-dev.
    Uses the global image_pipe loaded during initialization.
    FLUX.1-dev produces state-of-the-art image quality.
    """
    global image_pipe

    prompt = state.get("expanded_image_prompt")
    iteration = state.get("iteration_count", 0)

    if not prompt:
        print("[ERROR] No image prompt available.")
        return {"messages": [AIMessage(content="Error: No image prompt available.")]}

    print(f"\n[IMAGE] Using FLUX.1-dev (state-of-the-art quality)...")
    print(f"  {get_gpu_memory_usage()}")

    try:
        # Check if pipeline is loaded
        if image_pipe is None:
            print("  [IMAGE] Pipeline not loaded, loading now...")
            from diffusers import FluxPipeline
            image_pipe = FluxPipeline.from_pretrained(
                IMAGE_MODEL,
                torch_dtype=torch.bfloat16
            )

        # Move to GPU
        image_pipe.to("cuda")
        print(f"  [IMAGE] Model moved to GPU")
        print(f"  {get_gpu_memory_usage()}")

        print(f"[IMAGE] Generating image with FLUX.1-dev...")
        print(f"  Resolution: {IMAGE_WIDTH}x{IMAGE_HEIGHT}")
        print(f"  Steps: {IMAGE_STEPS}")
        print(f"  Prompt: {prompt[:100]}...")

        # FLUX.1-dev generation
        result = image_pipe(
            prompt=prompt,
            height=IMAGE_HEIGHT,
            width=IMAGE_WIDTH,
            num_inference_steps=IMAGE_STEPS,
            guidance_scale=3.5,  # FLUX.1-dev works well with 3-4
            generator=torch.Generator(device="cuda").manual_seed(42)
        )
        image = result.images[0]

        # Save image
        image_path = f"promotional_image_v{iteration + 1}.png"
        image.save(image_path)

        # Move back to CPU to free GPU memory for other models
        image_pipe.to("cpu")
        clear_gpu_memory()

        # Verify file
        import os
        if os.path.exists(image_path):
            file_size = os.path.getsize(image_path)
            print(f"[IMAGE] SUCCESS! Image saved: {image_path} ({file_size} bytes)")
        else:
            print(f"[ERROR] Failed to save image file")

        return {
            "generated_image": image,
            "messages": [AIMessage(content=f"Image generated: {image_path}")]
        }

    except Exception as e:
        import traceback
        if image_pipe is not None:
            try:
                image_pipe.to("cpu")
            except:
                pass
        clear_gpu_memory()
        print(f"[ERROR] Image generation failed: {str(e)}")
        traceback.print_exc()
        return {"messages": [AIMessage(content=f"Image error: {str(e)}")]}

### Tool 3: Generate Voice Over (Kokoro TTS)

In [ ]:
def generate_voiceover(state: AgentState) -> dict:
    """
    Generates voice-over audio using pre-loaded Kokoro-82M TTS.
    Uses the global tts_pipe loaded during initialization.
    Saves audio as WAV file for debugging.
    """
    global tts_pipe
    import soundfile as sf
    import numpy as np

    script = state.get("expanded_voiceover_script")
    iteration = state.get("iteration_count", 0)
    voice_style = state.get("voice_style", DEFAULT_VOICE)
    include_vo = state.get("include_voiceover", False)

    print(f"\n[DEBUG] generate_voiceover called:")
    print(f"  - include_voiceover: {include_vo}")
    print(f"  - script available: {bool(script)}")
    print(f"  - voice_style: {voice_style}")

    if not script:
        print("[ERROR] No voiceover script available.")
        return {
            "generated_audio_path": None,
            "messages": [AIMessage(content="Error: No voiceover script available.")]
        }

    if not include_vo:
        print("[INFO] Voiceover skipped (not requested).")
        return {
            "generated_audio_path": None,
            "messages": [AIMessage(content="Voiceover skipped.")]
        }

    print(f"\n[TTS] Using pre-loaded Kokoro TTS...")

    try:
        # Check if pipeline is loaded
        if tts_pipe is None:
            print("  [TTS] Pipeline not loaded, loading now...")
            from kokoro import KPipeline
            tts_pipe = KPipeline(lang_code='a')

        print(f"[TTS] Generating voiceover with voice: {voice_style}")
        print(f"[TTS] Script: '{script}'")

        # Generate audio
        generator = tts_pipe(
            script,
            voice=voice_style,
            speed=1.0
        )

        # Collect all audio chunks
        audio_chunks = []
        sample_rate = 24000  # Kokoro uses 24kHz

        for i, (gs, ps, audio) in enumerate(generator):
            audio_chunks.append(audio)
            print(f"  [TTS] Generated chunk {i+1}, shape: {audio.shape}")

        if not audio_chunks:
            print("[ERROR] No audio chunks generated!")
            return {
                "generated_audio_path": None,
                "messages": [AIMessage(content="Error: TTS generated no audio.")]
            }

        # Concatenate audio
        full_audio = np.concatenate(audio_chunks)
        print(f"[TTS] Full audio shape: {full_audio.shape}, duration: {len(full_audio)/sample_rate:.2f}s")

        # Save audio as WAV
        audio_path = f"voiceover_v{iteration + 1}.wav"
        sf.write(audio_path, full_audio, sample_rate)

        # Verify file was saved
        import os
        if os.path.exists(audio_path):
            file_size = os.path.getsize(audio_path)
            print(f"[TTS] SUCCESS! Voiceover saved: {audio_path} ({file_size} bytes)")
        else:
            print(f"[ERROR] Failed to save audio file: {audio_path}")
            return {
                "generated_audio_path": None,
                "messages": [AIMessage(content="Error: Failed to save audio file.")]
            }

        return {
            "generated_audio_path": audio_path,
            "messages": [AIMessage(content=f"Voiceover generated: {audio_path}")]
        }

    except Exception as e:
        import traceback
        print(f"[ERROR] TTS generation failed: {str(e)}")
        traceback.print_exc()
        return {
            "generated_audio_path": None,
            "messages": [AIMessage(content=f"TTS error: {str(e)}")]
        }

### Tool 4: Generate Video (LTX-Video)

In [ ]:
def generate_video(state: AgentState) -> dict:
    """
    Generates a promotional video using pre-loaded LTX-Video.
    Uses the global video_pipe loaded during initialization.
    LTX-Video is optimized for fast, real-time video generation.
    """
    global video_pipe
    from diffusers.utils import export_to_video

    prompt = state.get("expanded_video_prompt")
    iteration = state.get("iteration_count", 0)

    if not prompt:
        print("[ERROR] No video prompt available.")
        return {"messages": [AIMessage(content="Error: No video prompt available.")]}

    print(f"\n[VIDEO] Using LTX-Video (fast real-time generation)...")
    print(f"  {get_gpu_memory_usage()}")

    try:
        # Check if pipeline is loaded
        if video_pipe is None:
            print("  [VIDEO] Pipeline not loaded, loading now...")
            from diffusers import LTXPipeline
            video_pipe = LTXPipeline.from_pretrained(
                VIDEO_MODEL,
                torch_dtype=torch.bfloat16
            )

        # Move to GPU
        video_pipe.to("cuda")
        video_pipe.vae.enable_tiling()  # Memory optimization
        print(f"  [VIDEO] Model moved to GPU")
        print(f"  {get_gpu_memory_usage()}")

        print(f"[VIDEO] Generating video with LTX-Video...")
        print(f"  Resolution: {VIDEO_WIDTH}x{VIDEO_HEIGHT}")
        print(f"  Frames: {VIDEO_FRAMES} (~{VIDEO_FRAMES/VIDEO_FPS:.1f} seconds at {VIDEO_FPS} FPS)")
        print(f"  Prompt: {prompt[:150]}...")

        # Generate video - LTX-Video is much faster than CogVideoX
        start_time = time.time()

        negative_prompt = "worst quality, inconsistent motion, blurry, jittery, distorted"

        result = video_pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=VIDEO_WIDTH,
            height=VIDEO_HEIGHT,
            num_frames=VIDEO_FRAMES,
            num_inference_steps=50,
            decode_timestep=0.03,
            decode_noise_scale=0.025,
            generator=torch.Generator(device="cuda").manual_seed(42)
        )

        elapsed = time.time() - start_time
        print(f"  [VIDEO] Generation took {elapsed:.1f} seconds")

        # Export to video file
        video_frames = result.frames[0]
        video_path = f"promotional_video_v{iteration + 1}.mp4"
        export_to_video(video_frames, video_path, fps=VIDEO_FPS)

        # Move back to CPU to free GPU memory
        video_pipe.to("cpu")
        clear_gpu_memory()

        # Verify file
        import os
        if os.path.exists(video_path):
            file_size = os.path.getsize(video_path)
            print(f"[VIDEO] SUCCESS! Video saved: {video_path} ({file_size} bytes)")
        else:
            print(f"[ERROR] Failed to save video file")

        return {
            "generated_video_path": video_path,
            "messages": [AIMessage(content=f"Video generated: {video_path}")]
        }

    except Exception as e:
        import traceback
        if video_pipe is not None:
            try:
                video_pipe.to("cpu")
            except:
                pass
        clear_gpu_memory()
        print(f"[ERROR] Video generation failed: {str(e)}")
        traceback.print_exc()
        return {"messages": [AIMessage(content=f"Video error: {str(e)}")]}

### Tool 5: Combine Video and Audio (MoviePy)

In [ ]:
def combine_video_audio(state: AgentState) -> dict:
    """
    Combines generated video with voice-over audio using MoviePy.
    Compatible with MoviePy 2.0+
    Saves both the raw video and the combined video for debugging.
    """
    import os

    # MoviePy 2.0+ uses direct imports (not moviepy.editor)
    try:
        from moviepy import VideoFileClip, AudioFileClip
        moviepy_version = "2.0+"
    except ImportError:
        # Fallback for older MoviePy versions
        from moviepy.editor import VideoFileClip, AudioFileClip
        moviepy_version = "1.x"

    video_path = state.get("generated_video_path")
    audio_path = state.get("generated_audio_path")
    iteration = state.get("iteration_count", 0)
    include_vo = state.get("include_voiceover", False)

    print(f"\n[DEBUG] combine_video_audio called:")
    print(f"  - MoviePy version: {moviepy_version}")
    print(f"  - video_path: {video_path}")
    print(f"  - audio_path: {audio_path}")
    print(f"  - include_voiceover: {include_vo}")

    if not video_path:
        print("[ERROR] No video path provided.")
        return {
            "final_video_path": None,
            "messages": [AIMessage(content="Error: No video to combine.")]
        }

    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"[ERROR] Video file does not exist: {video_path}")
        return {
            "final_video_path": None,
            "messages": [AIMessage(content=f"Error: Video file not found: {video_path}")]
        }

    print(f"[COMBINE] Video file exists: {video_path} ({os.path.getsize(video_path)} bytes)")

    # Check if we should skip audio combination
    if not include_vo:
        print("[INFO] Voiceover not requested, using video as-is.")
        return {
            "final_video_path": video_path,
            "messages": [AIMessage(content="Video ready (voiceover not requested).")]
        }

    if not audio_path:
        print("[WARNING] No audio path provided, using video without voiceover.")
        return {
            "final_video_path": video_path,
            "messages": [AIMessage(content="Video ready (no audio path).")]
        }

    # Check if audio file exists
    if not os.path.exists(audio_path):
        print(f"[ERROR] Audio file does not exist: {audio_path}")
        return {
            "final_video_path": video_path,
            "messages": [AIMessage(content=f"Audio file not found, using video without audio.")]
        }

    print(f"[COMBINE] Audio file exists: {audio_path} ({os.path.getsize(audio_path)} bytes)")

    print("\n[COMBINE] Starting video and audio combination...")

    try:
        # Load video and audio
        print(f"[COMBINE] Loading video: {video_path}")
        video_clip = VideoFileClip(video_path)
        print(f"[COMBINE] Video loaded - duration: {video_clip.duration:.2f}s, size: {video_clip.size}")

        print(f"[COMBINE] Loading audio: {audio_path}")
        audio_clip = AudioFileClip(audio_path)
        print(f"[COMBINE] Audio loaded - duration: {audio_clip.duration:.2f}s")

        # Adjust audio duration to match video if needed
        if audio_clip.duration > video_clip.duration:
            print(f"[COMBINE] Trimming audio from {audio_clip.duration:.2f}s to {video_clip.duration:.2f}s")
            try:
                # MoviePy 2.0+ uses subclipped()
                audio_clip = audio_clip.subclipped(0, video_clip.duration)
            except AttributeError:
                # Fallback for older MoviePy versions
                audio_clip = audio_clip.subclip(0, video_clip.duration)

        # Set audio to video
        print("[COMBINE] Combining video with audio...")
        try:
            final_clip = video_clip.with_audio(audio_clip)
        except AttributeError:
            # Fallback for older MoviePy
            final_clip = video_clip.set_audio(audio_clip)

        # Export final video
        final_path = f"final_video_with_voiceover_v{iteration + 1}.mp4"
        print(f"[COMBINE] Exporting to: {final_path}")

        # MoviePy 2.0+ doesn't support verbose/logger args
        try:
            # Try MoviePy 2.0+ style first
            final_clip.write_videofile(
                final_path,
                codec='libx264',
                audio_codec='aac'
            )
        except TypeError:
            # Fallback for older MoviePy
            final_clip.write_videofile(
                final_path,
                codec='libx264',
                audio_codec='aac',
                verbose=False,
                logger=None
            )

        # Cleanup
        video_clip.close()
        audio_clip.close()
        final_clip.close()

        # Verify final file
        if os.path.exists(final_path):
            file_size = os.path.getsize(final_path)
            print(f"[COMBINE] SUCCESS! Final video saved: {final_path} ({file_size} bytes)")
        else:
            print(f"[ERROR] Final video file not created!")
            return {
                "final_video_path": video_path,
                "messages": [AIMessage(content="Error: Failed to create combined video.")]
            }

        return {
            "final_video_path": final_path,
            "messages": [AIMessage(content=f"Final video created: {final_path}")]
        }

    except Exception as e:
        import traceback
        print(f"[ERROR] Video/audio combination failed: {str(e)}")
        traceback.print_exc()
        # Fall back to video without audio
        return {
            "final_video_path": video_path,
            "messages": [AIMessage(content=f"Combine error, using video without audio: {str(e)}")]
        }

### Tool 6: Modify Prompt (Qwen2.5-7B Local)

In [ ]:
def modify_prompt(state: AgentState) -> dict:
    """
    Modifies existing prompts based on user feedback using Qwen2.5-7B (local).
    """
    feedback = state.get("feedback", "")
    current_image_prompt = state.get("expanded_image_prompt", "")
    current_video_prompt = state.get("expanded_video_prompt", "")
    current_voiceover = state.get("expanded_voiceover_script", "")

    system_prompt = f"""You are an expert at refining AI generation prompts based on user feedback.

Current image prompt:
{current_image_prompt}

Current video prompt:
{current_video_prompt}

Current voiceover script:
{current_voiceover}

User feedback: {feedback}

Modify the prompts to incorporate this feedback while preserving the core concept.

RULES:
- IMAGE: Do NOT request text rendering in the image
- VIDEO: Do NOT include any people, humans, faces, hands in the prompt
- VOICEOVER: Keep it under 40 words, natural and conversational

Format your response EXACTLY as:
IMAGE_PROMPT: [modified image prompt]
VIDEO_PROMPT: [modified video prompt - NO PEOPLE]
VOICEOVER_SCRIPT: [modified script]
"""

    print(f"\nModifying prompts based on feedback: '{feedback}'")

    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "Please modify the prompts based on the feedback."}
        ]

        response_text = generate_llm_response(messages)
        image_prompt, video_prompt, voiceover_script = parse_expanded_prompts(response_text)

        # Use original if parsing fails
        if not image_prompt:
            image_prompt = current_image_prompt
        if not video_prompt:
            video_prompt = current_video_prompt
        if not voiceover_script:
            voiceover_script = current_voiceover

        print("Prompts updated successfully.")

        return {
            "expanded_image_prompt": image_prompt,
            "expanded_video_prompt": video_prompt,
            "expanded_voiceover_script": voiceover_script,
            "iteration_count": state.get("iteration_count", 0) + 1,
            "messages": [AIMessage(content="Prompts updated based on feedback.")]
        }

    except Exception as e:
        print(f"Error modifying prompts: {str(e)}")
        return {
            "iteration_count": state.get("iteration_count", 0) + 1,
            "messages": [AIMessage(content=f"Modify error: {str(e)}")]
        }

## Node Function Definitions

In [ ]:
def get_user_input_node(state: AgentState) -> dict:
    """Initial node to get user's creative brief."""
    print("\n" + "="*60)
    print("   MARKETING CONTENT GENERATION AGENT (100% Open-Source)")
    print("   Powered by Qwen2.5, FLUX.1-dev, LTX-Video & Kokoro")
    print("   State-of-the-art quality - No API keys required!")
    print("="*60)

    user_input = input("\nDescribe what promotional material you need\n(e.g., 'coffee shop promotional material'): ").strip()

    if not user_input:
        user_input = "general business promotional material"

    return {
        "user_input": user_input,
        "iteration_count": 0,
        "is_satisfied": False,
        "include_voiceover": False,
        "voice_style": DEFAULT_VOICE,
        "messages": [HumanMessage(content=user_input)]
    }


def expand_prompt_node(state: AgentState) -> dict:
    """Node that calls the expand_prompt tool."""
    return expand_prompt(state)


def ask_generation_mode_node(state: AgentState) -> dict:
    """Ask user what type of content to generate, including voiceover option."""
    print("\n" + "-"*50)
    print("GENERATION OPTIONS")
    print("-"*50)
    print("\nWhat would you like to generate?")
    print("  1. Image only (FLUX.1-dev)")
    print("  2. Video only (LTX-Video, no voiceover)")
    print("  3. Video with voice-over (LTX-Video + Kokoro)")
    print("  4. Both image and video (with voice-over)")
    print("  5. Both image and video (no voice-over)")

    choice = input("\nEnter your choice (1-5): ").strip()

    mode_map = {
        "1": ("image", False),
        "2": ("video", False),
        "3": ("video", True),
        "4": ("both", True),
        "5": ("both", False)
    }
    mode, include_vo = mode_map.get(choice, ("both", True))

    print(f"\nSelected: {mode}" + (" with voiceover" if include_vo else ""))

    return {
        "generation_mode": mode,
        "include_voiceover": include_vo,
        "messages": [HumanMessage(content=f"Generate: {mode}, Voiceover: {include_vo}")]
    }

## Display Utilities

In [ ]:
def display_image(img, title="Generated Image"):
    """Display PIL image in notebook."""
    print(f"\n{title}:")
    try:
        plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.show()
    except Exception as e:
        print(f"Could not display image: {e}")
        print("Image saved to file.")


def display_video_in_notebook(video_path):
    """Display video in Jupyter/Colab notebook."""
    try:
        from IPython.display import Video, display
        display(Video(video_path, embed=True, height=400))
    except Exception as e:
        print(f"Could not display video inline: {e}")
        print(f"Video saved at: {video_path}")

In [ ]:
def generate_content_node(state: AgentState) -> dict:
    """
    Main generation node that routes to appropriate tools.
    Properly passes state through the pipeline with debugging.
    """
    import os

    mode = state.get("generation_mode", "both")
    include_vo = state.get("include_voiceover", False)
    results = {}

    print("\n" + "="*60)
    print("GENERATING CONTENT")
    print("="*60)
    print(f"\n[STATE DEBUG]")
    print(f"  - generation_mode: {mode}")
    print(f"  - include_voiceover: {include_vo}")
    print(f"  - voice_style: {state.get('voice_style', 'not set')}")
    print(f"  - voiceover_script: {state.get('expanded_voiceover_script', 'not set')[:50] if state.get('expanded_voiceover_script') else 'None'}...")

    # Step 1: Generate image if requested
    if mode in ["image", "both"]:
        print("\n" + "-"*50)
        print("[Step 1/4] GENERATING IMAGE")
        print("-"*50)
        img_result = generate_poster(state)
        results.update(img_result)
        if img_result.get("generated_image"):
            display_image(img_result["generated_image"], "Generated Promotional Image")

    # Step 2: Generate voiceover if requested (before video to save memory)
    if mode in ["video", "both"] and include_vo:
        print("\n" + "-"*50)
        print("[Step 2/4] GENERATING VOICEOVER")
        print("-"*50)
        # Create combined state with all values
        vo_state = {**state, **results}
        vo_result = generate_voiceover(vo_state)
        results.update(vo_result)

        # Debug: Check if audio was generated
        audio_path = vo_result.get("generated_audio_path")
        if audio_path and os.path.exists(audio_path):
            print(f"\n[CHECKPOINT] Voiceover file verified: {audio_path}")
        else:
            print(f"\n[WARNING] Voiceover may not have been generated properly")
    elif mode in ["video", "both"]:
        print("\n[Step 2/4] SKIPPING VOICEOVER (not requested)")

    # Step 3: Generate video if requested
    if mode in ["video", "both"]:
        print("\n" + "-"*50)
        print("[Step 3/4] GENERATING VIDEO")
        print("-"*50)
        vid_state = {**state, **results}
        vid_result = generate_video(vid_state)
        results.update(vid_result)

        # Debug: Check if video was generated
        video_path = vid_result.get("generated_video_path")
        if video_path and os.path.exists(video_path):
            print(f"\n[CHECKPOINT] Video file verified: {video_path}")

        # Step 4: Combine video and audio
        print("\n" + "-"*50)
        print("[Step 4/4] COMBINING VIDEO AND AUDIO")
        print("-"*50)

        # Create combined state with ALL accumulated results
        combine_state = {**state, **results}

        # Debug: Show what we're passing to combine
        print(f"\n[DEBUG] State being passed to combine_video_audio:")
        print(f"  - generated_video_path: {combine_state.get('generated_video_path')}")
        print(f"  - generated_audio_path: {combine_state.get('generated_audio_path')}")
        print(f"  - include_voiceover: {combine_state.get('include_voiceover')}")

        combine_result = combine_video_audio(combine_state)
        results.update(combine_result)

        # Display final video
        final_path = combine_result.get("final_video_path")
        if final_path and os.path.exists(final_path):
            print(f"\n[SUCCESS] Displaying final video: {final_path}")
            display_video_in_notebook(final_path)
        elif video_path:
            print(f"\n[INFO] Displaying raw video (no voiceover): {video_path}")
            display_video_in_notebook(video_path)

    # Print summary of generated files
    print("\n" + "="*60)
    print("GENERATION SUMMARY")
    print("="*60)
    print("\nGenerated files:")
    for key, value in results.items():
        if 'path' in key.lower() and value:
            if os.path.exists(value):
                size = os.path.getsize(value)
                print(f"  - {key}: {value} ({size} bytes)")
            else:
                print(f"  - {key}: {value} (FILE NOT FOUND!)")

    return results


def collect_feedback_node(state: AgentState) -> dict:
    """Collect user feedback on generated content."""
    iteration = state.get("iteration_count", 0)

    # Check iteration limit
    if iteration >= MAX_ITERATIONS:
        print(f"\nMaximum iterations ({MAX_ITERATIONS}) reached.")
        return {
            "is_satisfied": True,
            "messages": [AIMessage(content="Max iterations reached.")]
        }

    print("\n" + "-"*50)
    print("FEEDBACK")
    print("-"*50)
    print("\nAre you satisfied with the generated content?")
    print("  - Type 'yes' or 'done' if satisfied")
    print("  - Or provide feedback (e.g., 'make it more colorful', 'add more energy')")

    feedback = input("\nYour response: ").strip()

    if feedback.lower() in ["yes", "done", "satisfied", "looks good", "perfect", "ok", "okay", "y"]:
        return {
            "is_satisfied": True,
            "messages": [HumanMessage(content="Satisfied.")]
        }
    else:
        return {
            "feedback": feedback,
            "is_satisfied": False,
            "messages": [HumanMessage(content=f"Feedback: {feedback}")]
        }


def modify_prompt_node(state: AgentState) -> dict:
    """Node that calls the modify_prompt tool."""
    return modify_prompt(state)

## Graph Construction

In [ ]:
def build_agent_graph() -> StateGraph:
    """
    Builds the LangGraph agent with all nodes and edges.

    Graph Structure:

    START --> get_user_input --> expand_prompt --> ask_generation_mode
                                                         |
                                                         v
                                              generate_content
                                              (image -> voiceover -> video -> combine)
                                                         |
                                                         v
                                              collect_feedback
                                                    /    \
                                           [satisfied]  [feedback]
                                                /            \
                                              END        modify_prompt
                                                              |
                                                              v
                                                    generate_content (loop)
    """
    graph = StateGraph(AgentState)

    # Add all nodes
    graph.add_node("get_user_input", get_user_input_node)
    graph.add_node("expand_prompt", expand_prompt_node)
    graph.add_node("ask_generation_mode", ask_generation_mode_node)
    graph.add_node("generate_content", generate_content_node)
    graph.add_node("collect_feedback", collect_feedback_node)
    graph.add_node("modify_prompt", modify_prompt_node)

    # Add edges - linear flow
    graph.add_edge(START, "get_user_input")
    graph.add_edge("get_user_input", "expand_prompt")
    graph.add_edge("expand_prompt", "ask_generation_mode")
    graph.add_edge("ask_generation_mode", "generate_content")
    graph.add_edge("generate_content", "collect_feedback")

    # Conditional edge for feedback loop
    def route_after_feedback(state: AgentState) -> str:
        if state.get("is_satisfied", False):
            return "end"
        else:
            return "modify"

    graph.add_conditional_edges(
        "collect_feedback",
        route_after_feedback,
        {"end": END, "modify": "modify_prompt"}
    )

    # Loop back after modification
    graph.add_edge("modify_prompt", "generate_content")

    return graph.compile()

## Run the Agent

In [ ]:
def run_agent():
    """Main entry point for the marketing content generation agent."""
    import os

    # Build and compile the graph
    agent = build_agent_graph()

    # Initialize empty state
    initial_state = {
        "messages": [],
        "user_input": "",
        "feedback": None,
        "expanded_image_prompt": None,
        "expanded_video_prompt": None,
        "expanded_voiceover_script": None,
        "generation_mode": None,
        "include_voiceover": False,
        "voice_style": DEFAULT_VOICE,
        "generated_image": None,
        "generated_video_path": None,
        "generated_audio_path": None,
        "final_video_path": None,
        "iteration_count": 0,
        "is_satisfied": False,
    }

    try:
        # Run the agent graph
        final_state = agent.invoke(initial_state)

        # Final Summary
        print("\n" + "="*60)
        print("   GENERATION COMPLETE")
        print("="*60)
        print(f"\nTotal iterations: {final_state.get('iteration_count', 0) + 1}")

        # List all generated files
        print("\n" + "-"*40)
        print("GENERATED FILES:")
        print("-"*40)

        files_generated = []

        # Check for image
        if final_state.get("generated_image"):
            iteration = final_state.get("iteration_count", 0)
            img_path = f"promotional_image_v{iteration + 1}.png"
            if os.path.exists(img_path):
                files_generated.append(("Image", img_path, os.path.getsize(img_path)))

        # Check for voiceover audio
        audio_path = final_state.get("generated_audio_path")
        if audio_path and os.path.exists(audio_path):
            files_generated.append(("Voiceover (WAV)", audio_path, os.path.getsize(audio_path)))

        # Check for raw video
        video_path = final_state.get("generated_video_path")
        if video_path and os.path.exists(video_path):
            files_generated.append(("Raw Video (no audio)", video_path, os.path.getsize(video_path)))

        # Check for final video with voiceover
        final_path = final_state.get("final_video_path")
        if final_path and os.path.exists(final_path) and final_path != video_path:
            files_generated.append(("Final Video (with voiceover)", final_path, os.path.getsize(final_path)))

        if files_generated:
            for file_type, file_path, file_size in files_generated:
                size_str = f"{file_size / 1024:.1f} KB" if file_size < 1024*1024 else f"{file_size / (1024*1024):.1f} MB"
                print(f"  [{file_type}]")
                print(f"    Path: {file_path}")
                print(f"    Size: {size_str}")
                print()
        else:
            print("  No files were generated.")

        print("-"*40)
        print("\nThank you for using the Marketing Content Generator!")
        print("All files are saved in the current directory.")

        return final_state

    except KeyboardInterrupt:
        print("\n\nAgent interrupted by user.")
        return None
    except Exception as e:
        print(f"\n\nError: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

In [ ]:
# Run the agent
final_state = run_agent()